In [ ]:
import numpy as np
from scipy import stats

%run ../val-cruzada/pca.ipynb
%run ../knn/KNN.ipynb

Algortimo igual que carpeta knn pero en lugar de devolver el % devuelve el vector de las predicciones

In [ ]:
def knn_predict(X_train, X_test, y_train, k, distancia='coseno'):
    # Todas las distancias entre X_train y X_test
    if distancia == 'coseno':
        distancias = distCoseno(X_train, X_test)  # (entrenamiento x testeo)
    elif distancia == 'euclidiana':
        distancias = distancia_euclidea(X_train, X_test)  # (entrenamiento x testeo)

    # Obtener los índices de los k vecinos más cercanos para cada muestra de prueba
    k_indices = np.argsort(distancias, axis=0)[:k]  # (k, testeo)
    
    # Obtener las etiquetas de los k vecinos más cercanos
    k_labels = y_train[k_indices]  # (k, testeo)
    
    # Calcular la moda (etiqueta más frecuente) a lo largo de los k vecinos para cada muestra de prueba
    moda_labels, _ = stats.mode(k_labels, axis=0)
    
    return moda_labels.flatten() 

### Punto opcional: Variante de validación cruzada 

In [ ]:

def k_fold_cross_variante(x_train, y_train, distancia='coseno'):
    n = len(x_train)
    alto = n // 4
    indices = np.arange(n)
    
    # Almacena todas las predicciones
    todas_predicciones = np.zeros((len(intentos_p), len(intentos_k), n))

    for i in range(4):
        # Separar en índices de prueba y de entrenamiento para el fold actual
        test_indices = indices[i * alto: (i + 1) * alto]
        train_indices = np.concatenate((indices[:i * alto], indices[(i + 1) * alto:]))
        
        # Cargar los autovectores correspondientes a esta partición
        autovectores_train = np.loadtxt(f'../val-cruzada/particiones/autovectores_cov{i}.txt')
        
        for p_idx, p in enumerate(intentos_p):
            # Seleccionar los p autovectores y reducir la dimensionalidad
            x_autovectoresActual = autovectores_train[:, :p]
            x_train_centralizado = centralizar(x_train[train_indices])
            x_test_centralizado = centralizar(x_train[test_indices])
            
            # Reducir dimensionalidad
            x_auto_red = np.dot(x_train_centralizado, x_autovectoresActual)
            x_test_red = np.dot(x_test_centralizado, x_autovectoresActual)

            for k_idx, k in enumerate(intentos_k):
                # Realizar predicciones y almacenarlas en todas_predicciones
                predictions = knn_predict(x_auto_red, x_test_red, y_train[train_indices], k, distancia)
                todas_predicciones[p_idx, k_idx, test_indices] = predictions

    # Matriz de rendimientos
    matriz_rendimientos = np.zeros((len(intentos_p), len(intentos_k)))

    # Calcular el rendimiento global combinando todos los datos de prueba
    for p_idx in range(len(intentos_p)):
        for k_idx in range(len(intentos_k)):
            res = todas_predicciones[p_idx, k_idx, :] == y_train.flatten()
            matriz_rendimientos[p_idx, k_idx] = np.mean(res)
            
    return matriz_rendimientos
